In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.0/590.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.6/578.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfull

In [87]:
from sklearn.preprocessing import LabelEncoder

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-03-04 12:36:40--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2024-03-04 12:36:40 (9.90 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-03-04 12:36:40--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [135]:
MAX_LEN = 250
BATCH_SIZE = 16

In [136]:
train = pd.read_csv(train_file_path, sep='\t')
test = pd.read_csv(test_file_path, sep='\t')

In [137]:
columns = ['target', 'text']
train.columns = columns
test.columns = columns

In [138]:
encoder = LabelEncoder()
train['target'] = encoder.fit_transform(train['target'])
test['target'] = encoder.fit_transform(test['target'])

In [139]:
y_train = train.pop('target')
y_test = test.pop('target')

Preprocessing

In [140]:
data = pd.concat([train, test], axis = 0)

In [141]:
raw_data = ' '.join(data['text'].values.flatten())

In [142]:
tokens = tf.keras.preprocessing.text.text_to_word_sequence(raw_data)

In [143]:
vocab = sorted(set(tokens))
VOCAB_SIZE = len(vocab)

In [144]:
char2idx = {v : k for k,v in enumerate(vocab)}

In [145]:
def text_to_int(text) :
  text = tf.keras.preprocessing.text.text_to_word_sequence(text)
  return np.array([char2idx[c] for c in text])

In [146]:
x_train = train['text'].apply(text_to_int).values
x_test = test['text'].apply(text_to_int).values

In [147]:
x_train = tf.keras.utils.pad_sequences(x_train, MAX_LEN)
x_test = tf.keras.utils.pad_sequences(x_test, MAX_LEN)

In [154]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(VOCAB_SIZE, 64))
model.add(tf.keras.layers.LSTM(64))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [155]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [156]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 22s 150ms/step - accuracy: 0.8866 - loss: 0.3235 - val_accuracy: 0.9770 - val_loss: 0.0770
Epoch 2/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 148ms/step - accuracy: 0.9908 - loss: 0.0343 - val_accuracy: 0.9856 - val_loss: 0.0484
Epoch 3/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 149ms/step - accuracy: 0.9982 - loss: 0.0120 - val_accuracy: 0.9856 - val_loss: 0.0470
Epoch 4/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 140ms/step - accuracy: 0.9994 - loss: 0.0050 - val_accuracy: 0.9885 - val_loss: 0.0574
Epoch 5/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 150ms/step - accuracy: 0.9994 - loss: 0.0020 - val_accuracy: 0.9878 - val_loss: 0.0557


In [157]:
def encode_text(text) :
  tok = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tok = [char2idx[word] if word in char2idx else 0 for word in tok]
  return tf.keras.utils.pad_sequences([tok], MAX_LEN)[0]

In [158]:
def pred_idx(val, seuil = 0.5) :
  return tf.cast(tf.greater(val, seuil), tf.int32).numpy()[0,0]

In [160]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = encode_text(pred_text).reshape(-1, MAX_LEN)
  label = ['ham', 'spam']
  pred = model.predict(pred_text)

  idx = pred_idx(pred)

  return [pred[0][0], label[idx]]

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[0.00040152884, 'ham']


In [161]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
You passed the challenge. Great job!
